## MILP Formulation

This notebook aims to frame the problem in terms of Mixed Integer Linear Programming (MILP). The formulation is similar to the one provided in [this paper](https://ieeexplore.ieee.org/document/7519053). The tool to solve the optimization problem is [Gurobi](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html). `The notebook might fail to accurately and efficiently compute the solution, as the number of variables and constraints is huge, making the optimization problem particularly challenging.` In this case, a heuristic approach is commonly adopted, which are covered in [experiment.ipynb](experiment.ipynb).

In [ ]:
#!pip install gurobipy

In [2]:
import time
import random

from math import ceil

import numpy as np
import networkx as nx

import gurobipy as gp
from gurobipy import GRB

from algorithms.utils import k_shortest_paths

In [ ]:
seed = 9
random.seed(seed)
np.random.seed(seed)

In [ ]:
# Select the topology
TOPOLOGY = 'National' # 'Continental'
# Select spectrum partitioning
PARTITIONING = 'Hard' # 'Hard'
# Select the algorithm to be used
ALGORITHM = 'FF' # 'FF'

# Density of the network (wrt fully connected graph)
density = 0.5
# Capacity of the network links (THz)
CAPACITY = 4.4*(10**3) 
# Slice width (GHz) (used for section of 50 GHz (2 slices) or 75 GHz (3 slices))
slot_width = 25 
# Number of slots in the network
num_slots = int(CAPACITY/slot_width)

# Number of shortest paths to be considered
k = 4 

In [ ]:
####### TOPOLOGY PARAMETERS #######
if TOPOLOGY == 'Continental':
    print('Continental topology')
    # Continental topology
    Nodes = 7 # Number of nodes
    d_min, d_max = 400, 1100 # Min and max distances between nodes
    d_max_req, d_avg_req = 1000, 300 # Max and average distances (REQUIRED)
    # A requirement for the number of links
    req_links = lambda x: x>=20 
    
elif TOPOLOGY == 'National':
    print('National topology')
    # National topology
    Nodes = 4 # Number of nodes
    d_min, d_max = 150, 450 # Min and max distances between nodes
    d_max_req, d_avg_req = 400, 150 # Max and average distances (REQUIRED)    
    # A requirement for the number of links
    req_links = lambda x: x<=20 


In [ ]:
# Generate the distances between nodes as weights of the edges
distances = np.random.randint(d_min, d_max, size=(Nodes,Nodes))

# Generate the adjacency matrix
mask = np.random.choice([0, 1], size=(Nodes,Nodes), p=[1-density, density])
a = np.multiply(distances, mask)

# Force the matrix to be symmetric
A = ((a + a.T)/2).astype(int)
# Remove self loops
np.fill_diagonal(A,0)

# Create the graph
G = nx.DiGraph(A)
    
N, E = list(G.nodes()), list(G.edges())
weights = nx.get_edge_attributes(G,'weight')# set of lengths of each edge
weight_avg = np.mean(list(weights.values()))
weight_max = np.max(list(weights.values()))

# Check if the topology satisfies the requirements
print('Nodes:', N)
print('Edges:', E)
print('Is the number of links enough?', req_links(len(E)), f'(Number of links: {len(E)})')
print(f'Avg. distance: {weight_avg:.2f}km, Max. distance: {weight_max}km')
print(f'Avg. distance (REQUIRED): ~{d_avg_req} km, Max. distance (REQUIRED): ~{d_max_req}km')

In [ ]:
# Generate traffic matrix
demands = 150  # Number of demands
D = [] # Tuple of 3 values: (s, d), b 

for d in range(demands):
    s_d = random.sample(range(0, Nodes), 2)
    D.append((s_d[0], s_d[1] , np.random.choice([100,150,200,250,300,350,400])))
    
offered_traff = sum([D[d][2] for d in range(demands)])
print('Total offered traffic [Gbps]: ', offered_traff)

In [5]:
# Return 1 if path p uses link e
def r_pe(p, e):
    p_string = ''.join(str(i) for i in p)
    e_string = ''.join(str(i) for i in e)
    return e_string in p_string

# Set of pre-computed slots for demand d.
def C_d(d):
    b_d = D[d][2]

    b_0 = 200
    b_1 = 400
    
    # For every modulation
    for m in M: 
        # For m = 0 (modulation with 16QAM, bitrate 200Gbps, delta_f = 50GHz,  reach 900km)
        if m == 0: 
            l = ceil(b_d/b_0)
            # 2 slices of 25GHz = 50 GHz
            number_of_slices = (l)*2

            if number_of_slices == 2:
                L1 = S[0::2]
                L2 = S[1::2]
                slots_m0 = list(zip(L1, L2))

            if number_of_slices == 4:
                L1 = S[0::2]
                L2 = S[1::2]
                L3 = S[2::2]
                L4 = S[3::2]
                slots_m0 = list(zip(L1, L2, L3, L4))

        # For m = 0 (modulation with 16QAM, bitrate 200Gbps, delta_f = 50GHz,  reach 900km)
        if m == 1:
            l = ceil(b_d/b_1)
            # 2 slices of 25GHz = 50 GHz
            number_of_slices = (l)*3

            if number_of_slices == 3:
                L1 = S[0::3]
                L2 = S[1::3]
                L3 = S[2::3]
                slots_m1 = list(zip(L1, L2, L3))

            # WON'T HAPPEN, SINCE MAX BIT-RATE PER DEMAND IS LOWER THAN 400 Gbps
            if number_of_slices == 6:
                #print('4')
                L1 = S[0::3]
                L2 = S[1::3]
                L3 = S[2::3]
                L4 = S[3::3]
                L5 = S[4::3]
                L6 = S[5::3]
                slots_m1 = list(zip(L1, L2, L3, L4, L5, L6))
    return slots_m0 + slots_m1

# Equals to 1 if slot c ∈ C(d) is computed for modulation format m
def q_cm(c, m): 
    if m == 0 and (len(c)%2 == 0 or len(c)%4 == 0):
        return True
    if m == 1 and (len(c)%3 == 0 or len(c)%6 == 0):
        return True
    else: 
        return False 

# Equals to 1 if slot c uses slice s
def q_cs(c, s): 
    c_string = ''.join(str(i) for i in c)
    s_string = str(s)
    return s_string in c_string

In [ ]:
# Parameters for ILP
Capacity = 4.4*(10**3) #4.4THz

######################## Topology:
N = list(G.nodes()) # Set of nodes
E = list(G.edges()) # Set of enges
len_e = nx.get_edge_attributes(G,'weight') # Set of lengths of each edge

#################### Demands and Paths:
#D <-defined before
k = 3 # Shortest paths
P_d = [] # Subset of pre-computed paths for demand d
len_p = [] # Length of each precomputed path for the demand set

for d in range(demands):
    s = D[d][0]
    t = D[d][1]
    paths_list = k_shortest_paths(G, s, t,k, 'weight') # k shortest paths
    len_p.append([nx.path_weight(G, path, weight="weight") for path in paths_list]) # distances of the paths
    
    paths_tuples = []
    for p in paths_list:
        paths_tuples.append(tuple(p))
    P_d.append(paths_tuples)

#r_pe <-defined before

########################### Spectrum and Modulation Format
slice_width = 25 # GHz
S = list(range(0,int(Capacity/slice_width))) # Set of slices, each is 25GHz
print('Total # slices: ', len(S))

#C_d <-defined before

M = [0, 1] # Modulation formats (0 - 50GHz, 1 - 75 GHz)
len_m = [900, 600] # Reachability of the modulation formats

#q_cm <-defined before
#q_cs <-defined before

In [ ]:
####################################### ILP
model = gp.Model()
############################## Decision variables
#equal to 1 if demand d cannot be served
w = model.addVars(gp.tuplelist(range(demands)), vtype=GRB.BINARY)

#equal to 1 if demand d is routed through path p and slot c.
x = model.addVars(gp.tuplelist([(d, p, c) for d in range(demands) for p in P_d[d] for c in C_d(d)]), vtype=GRB.BINARY) 
                                

#number of times demand d modulated with m got regenerated
v = model.addVars(gp.tuplelist([(d, m) for d in range(demands) for m in M]), vtype=GRB.INTEGER, lb = 0)

#equal to 1 if demand d is routed through path p and slot c exceeds the reachability.
#v = {(d, p, c):model.add_var(var_type=BINARY,name = 'v_'+str(d)+'_'+str(p)+'_'+str(c)) 
#     for d in range(demands) for p in P_d[d] for c in C_d(d)}

In [9]:
# Objective function: minimize blocked traffic (first term, priority) 
# and the amound of regenerations (second term, less important wrt the first one)

model.setObjective(gp.quicksum(D[d][2]*w[d] for d in range(demands)) +
                              100*gp.quicksum(2*v[d,m] for d in range(demands) for m in M),GRB.MINIMIZE) 

In [ ]:
# Constraints

for d in range(demands):
    model.addConstr(gp.quicksum(x[d, p, c] for p in P_d[d] for c in C_d(d)) + w[d] == 1)

for d in range(demands):
    for m in M:
        model.addConstr(gp.quicksum(q_cm(c, m)*len_p[d][p_id] * x[d, p, c] 
                         for p_id, p in enumerate(P_d[d]) for c in C_d(d)) <= len_m[m]+ v[d,m]*len_m[m] )
        
for e in E:
    start_time1 = time.time()
    for s in S:
        model.addConstr(gp.quicksum(r_pe(p, e)*q_cs(c, s) * x[d, p, c] 
                                  for d in range(demands) for p in P_d[d] for c in C_d(d)) <= 1)
    end_time1 = time.time()
    print("Time to consider one link is is {:.2f} min".format((end_time1-start_time1)/60), sep='\n', end='\n', flush=True)

model.write('model.lp')

In [ ]:
start_time2 = time.time()
model.optimize()
end_time2 = time.time()
print("Time to optimize the model is {:.2f} min".format((end_time2-start_time2)/60))

In [ ]:
obj = model.getObjective()
print('Total offered traffic:', offered_traff)
print('Total dropped traffic:', obj.getValue())
print('Blocking probability: ',obj.getValue()/offered_traff)

In [ ]:
mod_var = model.getVars()
mod_var